# Laborator 3: Pregătirea și Curățarea Datelor

## Obiective
- Încărcarea și explorarea dataset-ului NSL-KDD
- Analiza exploratorie a datelor (EDA)
- Tratarea valorilor lipsă și a outlier-ilor
- Encoding pentru variabile categorice
- Normalizare/Standardizare
- Împărțirea în train/test sets

## Dataset: NSL-KDD
NSL-KDD este un dataset îmbunătățit pentru detectarea intruziunilor în rețea, derivat din KDD Cup 99.

**Clase de atacuri:**
- `normal` - trafic legitim
- `DoS` - Denial of Service
- `Probe` - Scanare și recunoaștere
- `R2L` - Remote to Local
- `U2R` - User to Root

## 1. Setup și Import Biblioteci

In [ ]:
# Instalare dependențe (rulați doar în Google Colab)
# !pip install pandas numpy scikit-learn matplotlib seaborn

In [ ]:
# Import biblioteci
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

# Setări afișare
pd.set_option('display.max_columns', 50)
plt.style.use('seaborn-v0_8-whitegrid')

print("Biblioteci încărcate cu succes!")

## 2. Încărcarea Datelor

Dataset-ul NSL-KDD poate fi descărcat de pe Kaggle sau de pe site-ul UNB.

**Opțiuni de încărcare:**
1. Upload manual în Colab
2. Descărcare directă din URL
3. Conectare la Google Drive

In [ ]:
# Definim numele coloanelor pentru NSL-KDD
# (dataset-ul original nu are header)

column_names = [
    'duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes',
    'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in',
    'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations',
    'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login',
    'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate',
    'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate',
    'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count',
    'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
    'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate',
    'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'label', 'difficulty'
]

print(f"Număr total de coloane: {len(column_names)}")
print(f"Coloane categorice: protocol_type, service, flag, label")
print(f"Coloane numerice: restul de {len(column_names) - 4} coloane")

In [ ]:
# OPȚIUNEA 1: Upload manual în Colab
# from google.colab import files
# uploaded = files.upload()

# OPȚIUNEA 2: Încărcare din fișier local sau URL
# Pentru acest laborator, vom folosi o versiune sintetică a dataset-ului
# În practică, descărcați de pe Kaggle: https://www.kaggle.com/datasets/hassan06/nslkdd

# Creăm un dataset sintetic pentru demonstrație
# (Înlocuiți cu calea reală către fișierul descărcat)

def create_sample_dataset(n_samples=10000):
    """Creează un dataset sintetic similar cu NSL-KDD pentru demonstrație."""
    np.random.seed(42)
    
    data = {
        'duration': np.random.exponential(100, n_samples),
        'protocol_type': np.random.choice(['tcp', 'udp', 'icmp'], n_samples, p=[0.7, 0.2, 0.1]),
        'service': np.random.choice(['http', 'ftp', 'smtp', 'ssh', 'dns', 'other'], n_samples),
        'flag': np.random.choice(['SF', 'S0', 'REJ', 'RSTR', 'SH'], n_samples),
        'src_bytes': np.random.exponential(1000, n_samples),
        'dst_bytes': np.random.exponential(2000, n_samples),
        'land': np.random.choice([0, 1], n_samples, p=[0.99, 0.01]),
        'wrong_fragment': np.random.choice([0, 1, 2, 3], n_samples, p=[0.95, 0.03, 0.01, 0.01]),
        'urgent': np.random.choice([0, 1], n_samples, p=[0.999, 0.001]),
        'hot': np.random.poisson(0.5, n_samples),
        'num_failed_logins': np.random.choice([0, 1, 2, 3], n_samples, p=[0.9, 0.07, 0.02, 0.01]),
        'logged_in': np.random.choice([0, 1], n_samples, p=[0.3, 0.7]),
        'num_compromised': np.random.poisson(0.1, n_samples),
        'root_shell': np.random.choice([0, 1], n_samples, p=[0.99, 0.01]),
        'su_attempted': np.random.choice([0, 1], n_samples, p=[0.995, 0.005]),
        'num_root': np.random.poisson(0.1, n_samples),
        'num_file_creations': np.random.poisson(0.2, n_samples),
        'num_shells': np.random.choice([0, 1], n_samples, p=[0.99, 0.01]),
        'num_access_files': np.random.poisson(0.1, n_samples),
        'num_outbound_cmds': np.zeros(n_samples),
        'is_host_login': np.random.choice([0, 1], n_samples, p=[0.999, 0.001]),
        'is_guest_login': np.random.choice([0, 1], n_samples, p=[0.99, 0.01]),
        'count': np.random.poisson(50, n_samples),
        'srv_count': np.random.poisson(30, n_samples),
        'serror_rate': np.random.beta(0.5, 5, n_samples),
        'srv_serror_rate': np.random.beta(0.5, 5, n_samples),
        'rerror_rate': np.random.beta(0.5, 10, n_samples),
        'srv_rerror_rate': np.random.beta(0.5, 10, n_samples),
        'same_srv_rate': np.random.beta(5, 1, n_samples),
        'diff_srv_rate': np.random.beta(1, 5, n_samples),
        'srv_diff_host_rate': np.random.beta(1, 5, n_samples),
        'dst_host_count': np.random.poisson(100, n_samples),
        'dst_host_srv_count': np.random.poisson(50, n_samples),
        'dst_host_same_srv_rate': np.random.beta(5, 1, n_samples),
        'dst_host_diff_srv_rate': np.random.beta(1, 5, n_samples),
        'dst_host_same_src_port_rate': np.random.beta(2, 3, n_samples),
        'dst_host_srv_diff_host_rate': np.random.beta(1, 5, n_samples),
        'dst_host_serror_rate': np.random.beta(0.5, 5, n_samples),
        'dst_host_srv_serror_rate': np.random.beta(0.5, 5, n_samples),
        'dst_host_rerror_rate': np.random.beta(0.5, 10, n_samples),
        'dst_host_srv_rerror_rate': np.random.beta(0.5, 10, n_samples),
        'label': np.random.choice(['normal', 'neptune', 'satan', 'ipsweep', 'portsweep', 
                                   'smurf', 'nmap', 'back', 'teardrop', 'warezclient'],
                                  n_samples, p=[0.5, 0.15, 0.05, 0.05, 0.05, 
                                               0.05, 0.05, 0.03, 0.04, 0.03]),
        'difficulty': np.random.randint(1, 22, n_samples)
    }
    
    return pd.DataFrame(data)

# Creăm dataset-ul
df = create_sample_dataset(10000)

print(f"Dataset încărcat: {df.shape[0]} rânduri x {df.shape[1]} coloane")

In [ ]:
# Primele 5 rânduri
df.head()

In [ ]:
# Informații despre dataset
df.info()

## 3. Exploratory Data Analysis (EDA)

Înainte de a preprocesa datele, trebuie să înțelegem:
- Distribuția claselor (echilibru/dezechilibru)
- Tipurile de date
- Valori lipsă
- Statistici descriptive
- Corelații între features

In [ ]:
# Statistici descriptive pentru coloanele numerice
df.describe()

In [ ]:
# Verificare valori lipsă
missing_values = df.isnull().sum()
print("Valori lipsă per coloană:")
print(missing_values[missing_values > 0])

if missing_values.sum() == 0:
    print("\nNu există valori lipsă în dataset! ✓")

In [ ]:
# Distribuția claselor (label)
print("Distribuția claselor:")
label_counts = df['label'].value_counts()
print(label_counts)

# Vizualizare
plt.figure(figsize=(12, 5))
label_counts.plot(kind='bar', color='steelblue', edgecolor='black')
plt.title('Distribuția Claselor în Dataset', fontsize=14)
plt.xlabel('Tip Trafic/Atac')
plt.ylabel('Număr Instanțe')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Mapare atacuri la categorii principale
attack_mapping = {
    'normal': 'normal',
    'neptune': 'DoS', 'smurf': 'DoS', 'back': 'DoS', 'teardrop': 'DoS', 
    'pod': 'DoS', 'land': 'DoS',
    'satan': 'Probe', 'ipsweep': 'Probe', 'nmap': 'Probe', 'portsweep': 'Probe',
    'warezclient': 'R2L', 'guess_passwd': 'R2L', 'warezmaster': 'R2L', 
    'imap': 'R2L', 'ftp_write': 'R2L',
    'buffer_overflow': 'U2R', 'rootkit': 'U2R', 'loadmodule': 'U2R', 'perl': 'U2R'
}

# Creăm o nouă coloană pentru categoria de atac
df['attack_category'] = df['label'].map(attack_mapping)

# Unele etichete pot să nu fie în mapping, le marcăm ca 'other'
df['attack_category'] = df['attack_category'].fillna('other')

print("\nDistribuția categoriilor de atacuri:")
print(df['attack_category'].value_counts())

In [ ]:
# Vizualizare categorii de atacuri
plt.figure(figsize=(10, 6))
category_counts = df['attack_category'].value_counts()
colors = ['#2ecc71', '#e74c3c', '#3498db', '#9b59b6', '#f39c12']
plt.pie(category_counts.values, labels=category_counts.index, autopct='%1.1f%%',
        colors=colors[:len(category_counts)], explode=[0.05]*len(category_counts))
plt.title('Distribuția Categoriilor de Atacuri', fontsize=14)
plt.show()

In [ ]:
# Distribuția protocoalelor
plt.figure(figsize=(8, 5))
df['protocol_type'].value_counts().plot(kind='bar', color=['#3498db', '#e74c3c', '#2ecc71'])
plt.title('Distribuția Protocoalelor', fontsize=14)
plt.xlabel('Protocol')
plt.ylabel('Număr Instanțe')
plt.xticks(rotation=0)
plt.show()

In [ ]:
# Corelații între features numerice (selectăm un subset pentru vizibilitate)
numeric_cols = ['duration', 'src_bytes', 'dst_bytes', 'count', 'srv_count', 
                'serror_rate', 'same_srv_rate', 'dst_host_count']

plt.figure(figsize=(10, 8))
correlation_matrix = df[numeric_cols].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            fmt='.2f', linewidths=0.5)
plt.title('Matricea de Corelație (Subset Features)', fontsize=14)
plt.tight_layout()
plt.show()

## 4. Preprocesarea Datelor

### 4.1 Separarea Features și Target

In [ ]:
# Creăm target binar: normal vs atac
df['is_attack'] = (df['label'] != 'normal').astype(int)

print("Distribuție target binar:")
print(df['is_attack'].value_counts())
print(f"\nProcentaj atacuri: {df['is_attack'].mean()*100:.1f}%")

In [ ]:
# Separăm features de target
# Eliminăm coloanele care nu sunt features
columns_to_drop = ['label', 'difficulty', 'attack_category', 'is_attack']

X = df.drop(columns=columns_to_drop)
y = df['is_attack']  # sau df['attack_category'] pentru clasificare multi-clasă

print(f"Shape features (X): {X.shape}")
print(f"Shape target (y): {y.shape}")

### 4.2 Encoding - Convertirea Variabilelor Categorice

Avem 3 coloane categorice:
- `protocol_type`: tcp, udp, icmp
- `service`: http, ftp, smtp, etc.
- `flag`: SF, S0, REJ, etc.

**Opțiuni de encoding:**
1. **Label Encoding** - transformă în numere (0, 1, 2, ...)
2. **One-Hot Encoding** - creează coloane binare pentru fiecare valoare

In [ ]:
# Identificăm coloanele categorice
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
print(f"Coloane categorice: {categorical_cols}")

for col in categorical_cols:
    print(f"\n{col}: {X[col].nunique()} valori unice")
    print(X[col].value_counts().head())

In [ ]:
# Metoda 1: Label Encoding
# Bun pentru arbori de decizie, dar poate introduce ordine falsă

X_label_encoded = X.copy()
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    X_label_encoded[col] = le.fit_transform(X[col])
    label_encoders[col] = le
    print(f"{col}: {dict(zip(le.classes_, le.transform(le.classes_)))}")

print("\nPrimele 5 rânduri după Label Encoding:")
X_label_encoded.head()

In [ ]:
# Metoda 2: One-Hot Encoding
# Nu introduce ordine falsă, dar crește numărul de coloane

X_onehot = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

print(f"Shape înainte de One-Hot: {X.shape}")
print(f"Shape după One-Hot: {X_onehot.shape}")
print(f"\nColoane noi create: {X_onehot.shape[1] - X.shape[1] + len(categorical_cols)}")

In [ ]:
# Vom folosi Label Encoding pentru simplitate
# (În practică, One-Hot e mai bun pentru rețele neurale și regresie logistică)

X_processed = X_label_encoded.copy()
print(f"\nShape final features: {X_processed.shape}")

### 4.3 Normalizare / Standardizare

De ce e important?
- Features cu valori mari domină calculele
- Algoritmii bazați pe distanță (KNN) sunt afectați
- Gradienții în rețele neurale converg mai bine

**Opțiuni:**
1. **StandardScaler**: (x - mean) / std → valori centrate pe 0
2. **MinMaxScaler**: (x - min) / (max - min) → valori în [0, 1]

In [ ]:
# Verificăm range-ul valorilor înainte de scalare
print("Range valori înainte de scalare:")
print(X_processed.describe().loc[['min', 'max', 'mean', 'std']])

In [ ]:
# StandardScaler - pentru majoritatea algoritmilor ML
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_processed)

# Convertim înapoi la DataFrame pentru vizualizare
X_scaled_df = pd.DataFrame(X_scaled, columns=X_processed.columns)

print("Range valori după StandardScaler:")
print(X_scaled_df.describe().loc[['min', 'max', 'mean', 'std']])

In [ ]:
# Alternativ: MinMaxScaler - pentru rețele neurale
minmax_scaler = MinMaxScaler()
X_minmax = minmax_scaler.fit_transform(X_processed)
X_minmax_df = pd.DataFrame(X_minmax, columns=X_processed.columns)

print("Range valori după MinMaxScaler:")
print(X_minmax_df.describe().loc[['min', 'max', 'mean', 'std']])

In [ ]:
# Vizualizare comparativă a distribuției unui feature înainte/după scalare
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

feature = 'src_bytes'

axes[0].hist(X_processed[feature], bins=50, color='steelblue', edgecolor='black')
axes[0].set_title(f'{feature} - Original')
axes[0].set_xlabel('Valoare')

axes[1].hist(X_scaled_df[feature], bins=50, color='green', edgecolor='black')
axes[1].set_title(f'{feature} - StandardScaler')
axes[1].set_xlabel('Valoare')

axes[2].hist(X_minmax_df[feature], bins=50, color='orange', edgecolor='black')
axes[2].set_title(f'{feature} - MinMaxScaler')
axes[2].set_xlabel('Valoare')

plt.tight_layout()
plt.show()

### 4.4 Train/Test Split

Împărțim datele în:
- **Train set** (80%): pentru antrenarea modelului
- **Test set** (20%): pentru evaluarea finală

**Important:** Folosim `stratify` pentru a păstra proporția claselor!

In [ ]:
# Împărțire stratificată
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled,  # Folosim datele scalate
    y,
    test_size=0.2,
    random_state=42,
    stratify=y  # Păstrează proporția claselor
)

print(f"Train set: {X_train.shape[0]} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Test set: {X_test.shape[0]} samples ({X_test.shape[0]/len(X)*100:.1f}%)")

In [ ]:
# Verificăm că proporțiile sunt păstrate
print("\nDistribuție clase în setul original:")
print(y.value_counts(normalize=True))

print("\nDistribuție clase în train set:")
print(pd.Series(y_train).value_counts(normalize=True))

print("\nDistribuție clase în test set:")
print(pd.Series(y_test).value_counts(normalize=True))

## 5. Salvarea Datelor Preprocesate

In [ ]:
# Salvăm datele preprocesate pentru laboratoarele următoare
import pickle

# Salvare în format numpy
np.save('X_train.npy', X_train)
np.save('X_test.npy', X_test)
np.save('y_train.npy', y_train)
np.save('y_test.npy', y_test)

# Salvare scaler pentru utilizare ulterioară
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# Salvare label encoders
with open('label_encoders.pkl', 'wb') as f:
    pickle.dump(label_encoders, f)

print("Date salvate cu succes!")
print("Fișiere create:")
print("  - X_train.npy, X_test.npy")
print("  - y_train.npy, y_test.npy")
print("  - scaler.pkl, label_encoders.pkl")

In [ ]:
# Salvare și în format CSV pentru backup
train_df = pd.DataFrame(X_train, columns=X_processed.columns)
train_df['target'] = y_train.values
train_df.to_csv('train_processed.csv', index=False)

test_df = pd.DataFrame(X_test, columns=X_processed.columns)
test_df['target'] = y_test.values
test_df.to_csv('test_processed.csv', index=False)

print("Fișiere CSV create: train_processed.csv, test_processed.csv")

## 6. Rezumat

În acest laborator am realizat:

1. **Încărcarea datelor** - Dataset NSL-KDD cu 41 features
2. **EDA** - Analiza distribuției claselor, protocoalelor, corelațiilor
3. **Encoding** - Label Encoding pentru variabile categorice
4. **Scalare** - StandardScaler pentru normalizarea valorilor
5. **Train/Test Split** - 80/20 cu stratificare

### Features finale:
- 38 features numerice (după encoding)
- Toate valorile scalate cu mean=0, std=1
- Target binar: 0=normal, 1=atac

### Următorul pas:
**Laborator 4** - Vom antrena modele de Machine Learning clasic (Decision Tree, Random Forest, KNN)

In [ ]:
# Sumar final
print("=" * 50)
print("SUMAR PREPROCESARE")
print("=" * 50)
print(f"\nDataset original: {df.shape[0]} samples x {df.shape[1]} features")
print(f"Features după encoding: {X_processed.shape[1]}")
print(f"\nTrain set: {X_train.shape}")
print(f"Test set: {X_test.shape}")
print(f"\nClase: 0=normal ({(y==0).sum()}), 1=atac ({(y==1).sum()})")
print(f"Raport atacuri: {y.mean()*100:.1f}%")